# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - Security Practices

**Subtopic** - Securing inter-service communication with signed tokens or API keys

**Use Case** - Implement a Python module that generates and verifies signed tokens for secure communication between two local services. This module should use the PyNaCl library for token generation and verification. It should also include functionality to securely store and retrieve these tokens using the local file system.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
pynacl==1.5.0
```


# Prompt

## **Problem Overview**

You are tasked with implementing a secure, independent Python module that manages cryptographically signed tokens for communication between trusted local services. The tokens must be signed and verified using the PyNaCl library, specifically Ed25519 keys.

To enhance robustness and long-term maintainability, the module must support **key rotation**, where new signing keys are periodically generated and older ones are retained for a grace period. Each key must have a **unique key ID (`kid`)**, and the system must verify tokens using the correct key based on this ID.

Additionally, your module must support **key revocation**, allowing administrators to revoke compromised or expired keys. Tokens signed with revoked keys must be rejected, even if their signatures are valid. The token should embed all necessary metadata (including `kid` and timestamp) in a base64-encoded signed message.

The module should persist keys and tokens on the local filesystem, and all file operations must be crash-resilient. It must expose a clean interface for token generation, verification, key rotation, and revocation, making it easy to integrate into larger service architectures.


## **Input Format**

* A `payload` (string or dictionary) for signing into a token.
* A `kid` (key ID) indicating which active signing key to use (optional; use latest if not provided).
* A signed token string (base64) for verification.
* Directory paths for storing key registry and tokens.
* A request to rotate or revoke keys.


## **Output Format**

* A base64-encoded signed token containing the payload and metadata.
* Boolean verification result, along with the extracted payload.
* File system storage of keys and tokens.
* Metadata files tracking active, historical, and revoked keys.


## **Examples**

### Example 1: Generate and verify a token with latest key

```python
>>> mgr = SecureTokenManager("key_store/")
>>> token = mgr.generate_token({"user": "alice"})
>>> valid, payload = mgr.verify_token(token)
>>> valid
True
>>> payload
{'user': 'alice'}
```

### Example 2: Generate a token with specific key ID

```python
>>> mgr = SecureTokenManager("key_store/")
>>> token = mgr.generate_token({"session": "x"}, kid="key-v2")
>>> mgr.verify_token(token)
(True, {'session': 'x'})
```

### Example 3: Revoke a key and reject future tokens

```python
>>> mgr.revoke_key("key-v1")
>>> mgr.verify_token(token_from_v1)
(False, None)
```


# Requirements


## **Explicit Requirements**

* Use **PyNaCl (Ed25519)** for key generation, token signing, and verification.
* Support **multiple signing keys**, each identified by a **unique `kid`**.
* Tokens must include the `kid` and be signed using the corresponding private key.
* During verification, dynamically **select the correct public key** based on `kid`.
* Provide functions to **rotate keys** and **generate a new default key**.
* Allow **revocation of specific keys**; revoked keys must not verify tokens.
* Tokens must support **UTF-8 string** and **JSON-serializable dictionary** payloads.
* Persist all keys and key metadata in a **local directory structure**.
* File writes must be **atomic and crash-resilient**.
* Signature failures, corrupted keys, and verification mismatches must be **logged and handled gracefully**.


## **Implicit Requirements**

* Tokens must embed the payload and metadata (including `kid`) inside the signed message.
* The key registry must distinguish between **active**, **revoked**, and **default** keys.
* Public keys must be **safely stored** for later verification.
* Module must be **thread-safe**, avoiding race conditions during key access or update.
* All classes and functions must be **modular and reusable**.
* The design should support **future extensions** (e.g., expiry, scope-based access).


## **Function Signature(s) and/or Class Definition(s)**

```python
class SecureTokenManager:
    def __init__(self, key_dir: str) -> None:
        """
        Initializes the key registry and loads or creates default keys.
        """

    def generate_token(self, payload: Union[str, Dict[str, Any]], kid: Optional[str] = None) -> str:
        """
        Signs a payload using the specified (or default) key and returns a base64-encoded token.
        """

    def verify_token(self, token: str) -> Tuple[bool, Optional[Union[str, Dict[str, Any]]]]:
        """
        Verifies a token using the appropriate key based on its embedded `kid`.
        Returns a tuple (is_valid, payload).
        """

    def rotate_key(self, new_kid: Optional[str] = None) -> str:
        """
        Generates a new key with a new `kid` (or auto-generated), sets it as default, and persists it.
        Returns the new key ID.
        """

    def revoke_key(self, kid: str) -> bool:
        """
        Marks a key as revoked. Tokens signed with this key will be rejected.
        Returns True if successful.
        """

    def list_keys(self) -> Dict[str, str]:
        """
        Returns a dictionary mapping key IDs to their status: active, revoked, or default.
        """
```


## **Edge Cases**

* Attempting to verify a token signed with a **revoked key**.
* Token includes a `kid` that doesn't exist or is not recognized.
* Corrupted or missing key files in the registry directory.
* Rotation invoked multiple times within a short time frame.
* Payload includes deeply nested JSON or non-serializable objects.
* Concurrent access by multiple processes attempting key rotation or revocation.


## **Constraints**

* Must be implemented in **Python 3.7+**.
* Use only **PyNaCl and standard libraries**.
* Keys and tokens must be **stored within the given `key_dir`**, with no outside writes.
* No external databases or services may be used.
* All persisted files must be written **atomically** to avoid corruption.
* The module must remain **self-contained**, reusable, and testable.

In [ ]:
# code

"""Token Manager module implemented with PyNaCl"""

import os
import json
import time
import uuid
import base64
import logging
import threading
from typing import Union, Dict, Any, Optional, Tuple
from pathlib import Path
import tempfile
import shutil

from nacl.signing import SigningKey, VerifyKey
from nacl.encoding import Base64Encoder
from nacl.exceptions import BadSignatureError, CryptoError

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class SecureTokenManager:
    """
    A secure token manager that supports key rotation, revocation, and cryptographic
    signing/verification using PyNaCl Ed25519 keys.
    """

    def __init__(self, key_dir: str) -> None:
        """
        Initializes the key registry and loads or creates default keys.

        Args:
            key_dir: Directory path for storing keys and metadata
        """
        self.key_dir = Path(key_dir)
        self.key_dir.mkdir(parents=True, exist_ok=True)

        # Thread safety
        self._lock = threading.RLock()

        # File paths
        self.registry_file = self.key_dir / "key_registry.json"
        self.keys_dir = self.key_dir / "keys"
        self.keys_dir.mkdir(exist_ok=True)

        # Initialize registry
        self._load_registry()

        # Create default key if no keys exist
        if not self.registry.get("keys"):
            self._create_default_key()

    def _load_registry(self) -> None:
        """Load the key registry from disk or create a new one."""
        try:
            if self.registry_file.exists():
                with open(self.registry_file, 'r') as f:
                    self.registry = json.load(f)
            else:
                self.registry = {
                    "keys": {},
                    "default_key": None,
                    "version": 1
                }
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Failed to load registry: {e}")
            self.registry = {
                "keys": {},
                "default_key": None,
                "version": 1
            }

    def _save_registry(self) -> None:
        """Atomically save the key registry to disk."""
        try:
            # Write to temporary file first
            with tempfile.NamedTemporaryFile(
                mode='w',
                dir=self.key_dir,
                delete=False,
                suffix='.tmp'
            ) as tmp_file:
                json.dump(self.registry, tmp_file, indent=2)
                tmp_file.flush()
                os.fsync(tmp_file.fileno())
                temp_path = tmp_file.name

            # Atomic rename
            shutil.move(temp_path, self.registry_file)

        except IOError as e:
            logger.error(f"Failed to save registry: {e}")
            # Clean up temp file if it exists
            if 'temp_path' in locals() and os.path.exists(temp_path):
                os.unlink(temp_path)
            raise

    def _generate_key_id(self) -> str:
        """Generate a unique key ID."""
        return f"key-{uuid.uuid4().hex[:8]}"

    def _create_default_key(self) -> str:
        """Create and set a default signing key."""
        kid = self._generate_key_id()
        return self._create_key(kid, set_as_default=True)

    def _create_key(self, kid: str, set_as_default: bool = False) -> str:
        """
        Create a new signing key and persist it.

        Args:
            kid: Key ID for the new key
            set_as_default: Whether to set this as the default key

        Returns:
            The key ID of the created key
        """
        # Generate new key pair
        signing_key = SigningKey.generate()
        verify_key = signing_key.verify_key

        # Key metadata
        key_metadata = {
            "kid": kid,
            "status": "active",
            "created_at": time.time(),
            "public_key": verify_key.encode(Base64Encoder).decode('utf-8'),
            "revoked_at": None
        }

        # Store private key separately
        private_key_path = self.keys_dir / f"{kid}.priv"
        public_key_path = self.keys_dir / f"{kid}.pub"

        try:
            # Atomically write private key
            with tempfile.NamedTemporaryFile(
                mode='wb',
                dir=self.keys_dir,
                delete=False,
                suffix='.tmp'
            ) as tmp_file:
                tmp_file.write(signing_key.encode())
                tmp_file.flush()
                os.fsync(tmp_file.fileno())
                temp_priv_path = tmp_file.name

            # Atomically write public key
            with tempfile.NamedTemporaryFile(
                mode='wb',
                dir=self.keys_dir,
                delete=False,
                suffix='.tmp'
            ) as tmp_file:
                tmp_file.write(verify_key.encode())
                tmp_file.flush()
                os.fsync(tmp_file.fileno())
                temp_pub_path = tmp_file.name

            # Atomic moves
            shutil.move(temp_priv_path, private_key_path)
            shutil.move(temp_pub_path, public_key_path)

            # Update registry
            self.registry["keys"][kid] = key_metadata
            if set_as_default:
                self.registry["default_key"] = kid

            self._save_registry()

            logger.info(f"Created new key: {kid}")
            return kid

        except Exception as e:
            logger.error(f"Failed to create key {kid}: {e}")
            # Clean up temp files
            for temp_path in [temp_priv_path, temp_pub_path]:
                if 'temp_path' in locals() and os.path.exists(temp_path):
                    os.unlink(temp_path)
            raise

    def _load_signing_key(self, kid: str) -> Optional[SigningKey]:
        """Load a signing key from disk."""
        try:
            private_key_path = self.keys_dir / f"{kid}.priv"
            if not private_key_path.exists():
                return None

            with open(private_key_path, 'rb') as f:
                return SigningKey(f.read())

        except Exception as e:
            logger.error(f"Failed to load signing key {kid}: {e}")
            return None

    def _load_verify_key(self, kid: str) -> Optional[VerifyKey]:
        """Load a verify key from disk."""
        try:
            public_key_path = self.keys_dir / f"{kid}.pub"
            if not public_key_path.exists():
                return None

            with open(public_key_path, 'rb') as f:
                return VerifyKey(f.read())

        except Exception as e:
            logger.error(f"Failed to load verify key {kid}: {e}")
            return None

    def generate_token(self, payload: Union[str, Dict[str, Any]], kid: Optional[str] = None) -> str:
        """
        Signs a payload using the specified (or default) key and returns a base64-encoded token.

        Args:
            payload: The data to sign (string or dictionary)
            kid: Key ID to use for signing (uses default if None)

        Returns:
            Base64-encoded signed token

        Raises:
            ValueError: If key is not found or revoked
            RuntimeError: If signing fails
        """
        with self._lock:
            # Determine which key to use
            if kid is None:
                kid = self.registry.get("default_key")
                if kid is None:
                    raise ValueError("No default key available")

            # Check if key exists and is active
            key_info = self.registry["keys"].get(kid)
            if key_info is None:
                raise ValueError(f"Key {kid} not found")

            if key_info["status"] == "revoked":
                raise ValueError(f"Key {kid} is revoked")

            # Load signing key
            signing_key = self._load_signing_key(kid)
            if signing_key is None:
                raise RuntimeError(f"Failed to load signing key {kid}")

            # Prepare token data
            token_data = {
                "kid": kid,
                "payload": payload,
                "timestamp": time.time(),
                "version": 1
            }

            try:
                # Serialize and sign
                message = json.dumps(token_data, separators=(',', ':')).encode('utf-8')
                signed_message = signing_key.sign(message)

                # Return base64-encoded signed message
                return base64.b64encode(signed_message).decode('utf-8')

            except (TypeError, ValueError) as e:
                logger.error(f"Failed to serialize payload for key {kid}: {e}")
                raise ValueError(f"Payload is not JSON serializable: {e}")
            except Exception as e:
                logger.error(f"Failed to sign token with key {kid}: {e}")
                raise RuntimeError(f"Token signing failed: {e}")

    def verify_token(self, token: str) -> Tuple[bool, Optional[Union[str, Dict[str, Any]]]]:
        """
        Verifies a token using the appropriate key based on its embedded `kid`.
        Returns a tuple (is_valid, payload).

        Args:
            token: Base64-encoded signed token to verify

        Returns:
            Tuple of (is_valid, payload or None)
        """
        with self._lock:
            try:
                # Decode token
                signed_message = base64.b64decode(token.encode('utf-8'))

                # We need to extract the kid first, but the message is signed
                # We'll try to verify with each available key until one works
                # or we can parse the message to get the kid

                # Try to find the correct key by attempting verification
                for kid, key_info in self.registry["keys"].items():
                    if key_info["status"] == "revoked":
                        continue

                    verify_key = self._load_verify_key(kid)
                    if verify_key is None:
                        continue

                    try:
                        # Attempt to verify and extract message
                        message = verify_key.verify(signed_message)
                        token_data = json.loads(message.decode('utf-8'))

                        # Check if this is the correct key
                        if token_data.get("kid") == kid:
                            # Additional validation
                            if key_info["status"] == "revoked":
                                logger.warning(f"Token signed with revoked key {kid}")
                                return False, None

                            payload = token_data.get("payload")
                            return True, payload

                    except (BadSignatureError, json.JSONDecodeError, CryptoError):
                        continue
                    except Exception as e:
                        logger.error(f"Unexpected error verifying with key {kid}: {e}")
                        continue

                return False, None

            except Exception as e:
                logger.error(f"Token verification failed: {e}")
                return False, None

    def rotate_key(self, new_kid: Optional[str] = None) -> str:
        """
        Generates a new key with a new `kid` (or auto-generated), sets it as default, and persists it.
        Returns the new key ID.

        Args:
            new_kid: Specific key ID to use (auto-generated if None)

        Returns:
            The new key ID

        Raises:
            ValueError: If key ID already exists
        """
        with self._lock:
            if new_kid is None:
                new_kid = self._generate_key_id()

            # Check if key ID already exists
            if new_kid in self.registry["keys"]:
                raise ValueError(f"Key ID {new_kid} already exists")

            # Create new key and set as default
            return self._create_key(new_kid, set_as_default=True)

    def revoke_key(self, kid: str) -> bool:
        """
        Marks a key as revoked. Tokens signed with this key will be rejected.
        Returns True if successful.

        Args:
            kid: Key ID to revoke

        Returns:
            True if successful, False otherwise
        """
        with self._lock:
            if kid not in self.registry["keys"]:
                logger.warning(f"Attempted to revoke non-existent key: {kid}")
                return False

            key_info = self.registry["keys"][kid]
            if key_info["status"] == "revoked":
                logger.info(f"Key {kid} already revoked")
                return True

            # Mark as revoked
            key_info["status"] = "revoked"
            key_info["revoked_at"] = time.time()

            # If this was the default key, we need to set a new default
            if self.registry["default_key"] == kid:
                # Find another active key to set as default
                new_default = None
                for k, v in self.registry["keys"].items():
                    if v["status"] == "active" and k != kid:
                        new_default = k
                        break

                if new_default:
                    self.registry["default_key"] = new_default
                    logger.info(f"Set new default key: {new_default}")
                else:
                    # No active keys left, create a new one
                    new_default = self._create_key(self._generate_key_id(), set_as_default=True)
                    logger.info(f"Created new default key: {new_default}")

            try:
                self._save_registry()
                logger.info(f"Revoked key: {kid}")
                return True
            except Exception as e:
                logger.error(f"Failed to save registry after revoking key {kid}: {e}")
                return False

    def list_keys(self) -> Dict[str, str]:
        """
        Returns a dictionary mapping key IDs to their status: active, revoked, or default.

        Returns:
            Dictionary mapping key IDs to their status
        """
        with self._lock:
            result = {}
            default_key = self.registry.get("default_key")

            for kid, key_info in self.registry["keys"].items():
                status = key_info["status"]
                if kid == default_key and status == "active":
                    status = "default"
                result[kid] = status

            return result


In [ ]:
# tests

import os
import shutil
import unittest
from base64 import b64decode
from tempfile import mkdtemp
from main import SecureTokenManager


class TestSecureTokenManager(unittest.TestCase):
    """Unit tests for SecureTokenManager covering full functionality."""

    def setUp(self):
        self.key_dir = mkdtemp()
        self.mgr = SecureTokenManager(self.key_dir)

    def tearDown(self):
        shutil.rmtree(self.key_dir)

    def test_generate_and_verify_token_with_default_key(self):
        token = self.mgr.generate_token({"user": "alice"})
        valid, payload = self.mgr.verify_token(token)
        self.assertTrue(valid)
        self.assertEqual(payload, {"user": "alice"})

    def test_generate_token_with_specific_kid(self):
        kid = self.mgr.rotate_key("key-v2")
        token = self.mgr.generate_token({"x": "1"}, kid=kid)
        valid, payload = self.mgr.verify_token(token)
        self.assertTrue(valid)
        self.assertEqual(payload, {"x": "1"})

    def test_verify_token_signed_with_revoked_key_fails(self):
        # Capture the current default key ID
        keys = self.mgr.list_keys()
        kid = next(k for k, status in keys.items() if status == "default")

        token = self.mgr.generate_token("secure-msg", kid=kid)
        self.mgr.revoke_key(kid)

        valid, payload = self.mgr.verify_token(token)
        self.assertFalse(valid)
        self.assertIsNone(payload)

    def test_rotate_key_creates_unique_key_id(self):
        kid1 = self.mgr.rotate_key("custom-key1")
        kid2 = self.mgr.rotate_key("custom-key2")
        self.assertNotEqual(kid1, kid2)
        self.assertIn(kid2, self.mgr.list_keys())

    def test_list_keys_shows_correct_statuses(self):
        kid1 = self.mgr.rotate_key("v1")
        kid2 = self.mgr.rotate_key("v2")
        self.mgr.revoke_key(kid1)
        keys = self.mgr.list_keys()
        self.assertEqual(keys[kid1], "revoked")
        self.assertEqual(keys[kid2], "default")

    def test_token_with_nonexistent_kid_fails_verification(self):
        fake_token = self._forge_token_with_kid("fake-kid")
        valid, _ = self.mgr.verify_token(fake_token)
        self.assertFalse(valid)

    def test_rotate_multiple_keys_quickly(self):
        kids = [self.mgr.rotate_key(f"key{i}") for i in range(5)]
        listed = self.mgr.list_keys()
        self.assertEqual(len(kids), 5)
        for k in kids:
            self.assertIn(k, listed)

    def test_deeply_nested_json_payload(self):
        payload = {"a": {"b": {"c": {"d": "deep"}}}}
        token = self.mgr.generate_token(payload)
        valid, data = self.mgr.verify_token(token)
        self.assertTrue(valid)
        self.assertEqual(data, payload)

    def test_revoking_key_returns_true_on_success(self):
        kid = self.mgr.rotate_key("rev-key")
        result = self.mgr.revoke_key(kid)
        self.assertTrue(result)

    def test_revoke_already_revoked_key_is_idempotent(self):
        kid = self.mgr.rotate_key("dup-revoke")
        self.mgr.revoke_key(kid)
        result = self.mgr.revoke_key(kid)
        self.assertTrue(result)

    def test_token_with_string_payload_verifies(self):
        token = self.mgr.generate_token("hello world")
        valid, payload = self.mgr.verify_token(token)
        self.assertTrue(valid)
        self.assertEqual(payload, "hello world")

    def test_token_contains_embedded_metadata(self):
        token = self.mgr.generate_token({"id": 99})
        raw = b64decode(token)
        self.assertIn(b'"kid":', raw)
        self.assertIn(b'"payload":', raw)

    def test_key_directory_is_persisted_correctly(self):
        token = self.mgr.generate_token("persist-test")
        files = os.listdir(self.key_dir)
        self.assertGreater(len(files), 0)
        self.assertTrue(any("key" in f or f.endswith(".priv") or
                        f.endswith(".pub") for f in files))

    def test_concurrent_key_rotations_create_unique_keys(self):
        kids = set()
        for i in range(10):
            kids.add(self.mgr.rotate_key(f"rotate-{i}"))
        self.assertEqual(len(kids), 10)

    def test_token_from_old_active_key_still_valid(self):
        kid1 = self.mgr.rotate_key("old-key")
        token = self.mgr.generate_token("data", kid=kid1)
        self.mgr.rotate_key("new-key")
        valid, payload = self.mgr.verify_token(token)
        self.assertTrue(valid)
        self.assertEqual(payload, "data")

    def test_corrupted_token_string_fails_gracefully(self):
        token = "not-a-real-token"
        valid, payload = self.mgr.verify_token(token)
        self.assertFalse(valid)
        self.assertIsNone(payload)

    def test_key_files_are_atomic_and_exist(self):
        self.mgr.generate_token("check-file-write")
        key_files = os.listdir(self.key_dir)
        paths = [os.path.join(self.key_dir, f) for f in key_files]
        self.assertTrue(any(os.path.isfile(p) for p in paths))

    def _extract_kid_from_token(self, token):
        import json
        raw = b64decode(token)
        return json.loads(raw)["kid"]

    def _forge_token_with_kid(self, kid):
        import json
        from base64 import b64encode
        fake_data = {"kid": kid, "payload": "x"}
        msg = json.dumps(fake_data).encode()
        return b64encode(msg).decode()


if __name__ == '__main__':
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/8afdfde1-0c03-4e5b-952e-71af74549775?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F73855%2Fview

#### Model code:

```python
import os
import json
import time
import base64
import tempfile
import shutil
from typing import Optional, Union, Dict, Tuple, Any
from nacl.signing import SigningKey, VerifyKey
from nacl.exceptions import BadSignatureError

def _atomic_write_file(file_path: str, data: bytes) -> None:
    """
    Writes data to a temporary file first, then renames it to the target path.
    This ensures atomicity and reduces risk of corruption on crashes.
    """
    dir_name = os.path.dirname(file_path)
    base_name = os.path.basename(file_path)

    # Create a temporary file in the same directory
    with tempfile.NamedTemporaryFile(dir=dir_name, delete=False) as tmp_file:
        tmp_file.write(data)
        temp_name = tmp_file.name

    # Atomically rename the temp file to the final destination
    os.replace(temp_name, os.path.join(dir_name, base_name))


class SecureTokenManager:
    REGISTRY_FILENAME = "registry.json"

    def __init__(self, key_dir: str) -> None:
        """
        Initializes the key manager, loading or creating keys in key_dir.
        """
        self.key_dir = os.path.abspath(key_dir)
        os.makedirs(self.key_dir, exist_ok=True)
        self.registry_path = os.path.join(self.key_dir, self.REGISTRY_FILENAME)

        # Load or create registry
        self._registry = self._load_registry()
        if not self._registry:
            # Create a brand-new registry structure
            self._registry = {
                "default_kid": None,
                "keys": {}  # kid -> { public_path, private_path, status }
            }
            # Create an initial default key
            initial_kid = self.rotate_key(new_kid="key-v1")
            self._registry["default_kid"] = initial_kid
            self._write_registry()

    def generate_token(self,
                       payload: Union[str, Dict[str, Any]],
                       kid: Optional[str] = None) -> str:
        """
        Signs the given payload (str/dict) with the specified or default key,
        embedding the kid and timestamp. Returns a base64-encoded signed token.
        """
        if isinstance(payload, dict):
            payload_data = payload
        else:
            payload_data = {"value": payload}

        if kid is None:
            # Use the default key if no kid is passed
            kid = self._registry.get("default_kid")

        # Fetch key info
        key_info = self._registry["keys"].get(kid)
        if not key_info:
            raise ValueError(f"Key ID {kid} not found in registry.")

        if key_info["status"] == "revoked":
            raise ValueError(f"Cannot sign with revoked key: {kid}")

        private_path = key_info["private_path"]
        priv_key_bytes = self._read_file_bytes(os.path.join(self.key_dir, private_path))
        signing_key = SigningKey(priv_key_bytes)

        # Build the token content (JSON)
        now = int(time.time())
        token_content = {
            "kid": kid,
            "ts": now,
            "data": payload_data
        }
        message = json.dumps(token_content, separators=(',', ':')).encode("utf-8")

        # Sign message
        signed_message = signing_key.sign(message)
        # Return base64 encoding of the signed data
        token_b64 = base64.urlsafe_b64encode(signed_message).decode("utf-8")
        return token_b64

    def verify_token(self, token: str) -> Tuple[bool, Optional[Union[str, Dict[str, Any]]]]:
        """
        Verifies a base64-encoded token. Returns (True, payload) if valid,
        or (False, None) if invalid or revoked.
        """
        try:
            signed_data = base64.urlsafe_b64decode(token.encode("utf-8"))
        except Exception:
            # Corrupted base64
            return (False, None)

        # The signed data includes the original message + signature appended
        # We first parse out the embedded JSON to find the kid, then verify signature
        # with the correct public key.
        # In PyNaCl, to reliably decode the original message, we need the public key.
        # We'll attempt a small trick: parse kid from the raw message portion, but since
        # we can't trust it without verification, we handle carefully.

        # We can try each active (or known) key in the registry, but that's inefficient.
        # Instead, we can use the standard approach with Ed25519: we must know which key to try.
        # So we do a small guess: the 'kid' should be in the last portion of the message.
        # Alternatively, we can store kid in the message, parse it after verifying each
        # candidate. But the instructions specify that "Tokens must include the kid and
        # be signed using the corresponding private key," so let's do it in two steps:
        #
        # 1) Try each known (non-revoked) key's public key to see if signature verification
        #    works. If it works, parse the message, check the kid match.
        # 2) Return success if the correct kid is found and the key is not revoked.
        # 3) Otherwise fail.
        #
        # This approach allows verifying even if we don't trust the extracted kid from the token.

        for candidate_kid, info in self._registry["keys"].items():
            if info["status"] == "revoked":
                continue  # skip revoked keys

            pub_key_bytes = self._read_file_bytes(os.path.join(self.key_dir, info["public_path"]))
            verify_key = VerifyKey(pub_key_bytes)
            try:
                # If the signature is valid, this returns the original message
                msg = verify_key.verify(signed_data)
                # Now parse the message
                parsed = json.loads(msg.decode("utf-8"))
                # Check that the kid in the message matches the candidate_kid
                actual_kid = parsed.get("kid")
                if actual_kid == candidate_kid:
                    # Found a valid signature with matching kid
                    # Final check: ensure key isn't revoked. (We skip if above)
                    data = parsed.get("data")
                    # Return the payload in a consistent format
                    if isinstance(data, dict) and "value" in data and len(data) == 1:
                        # This was originally a string payload
                        return (True, data["value"])
                    return (True, data)
            except BadSignatureError:
                # Not valid for this key, move on
                pass
            except Exception:
                # Some parse error or unexpected decoding failure
                pass

        # If we reach here, no valid signature was found
        return (False, None)

    def rotate_key(self, new_kid: Optional[str] = None) -> str:
        """
        Generates a new Ed25519 key pair. If new_kid is None, auto-generate one.
        Marks it as the new default, writes to the registry, and returns the kid.
        """
        if not new_kid:
            new_kid = f"key-{int(time.time())}"  # simplistic ID

        # Create Ed25519 key pair
        signing_key = SigningKey.generate()
        verify_key = signing_key.verify_key

        # Write key pair to disk
        priv_path = f"{new_kid}-private.key"
        pub_path = f"{new_kid}-public.key"

        _atomic_write_file(os.path.join(self.key_dir, priv_path), signing_key.encode())
        _atomic_write_file(os.path.join(self.key_dir, pub_path), verify_key.encode())

        # Register it in the local registry
        self._registry["keys"][new_kid] = {
            "private_path": priv_path,
            "public_path": pub_path,
            "status": "active"
        }
        # Mark it as default
        self._registry["default_kid"] = new_kid
        self._write_registry()
        return new_kid

    def revoke_key(self, kid: str) -> bool:
        """
        Marks a key as revoked in the registry. Returns True if successful;
        returns False if the kid doesn't exist or was already revoked.
        """
        key_info = self._registry["keys"].get(kid)
        if not key_info:
            return False
        if key_info["status"] == "revoked":
            return False

        key_info["status"] = "revoked"

        # If this key was default, remove the default
        if self._registry["default_kid"] == kid:
            self._registry["default_kid"] = None

        self._write_registry()
        return True

    def list_keys(self) -> Dict[str, str]:
        """
        Returns a dictionary of kid -> status for all known keys.
        """
        return {k: v["status"] for k, v in self._registry["keys"].items()}

    def _load_registry(self) -> Dict[str, Any]:
        """
        Reads the registry.json if present, returns dict or None if not found.
        """
        registry_data = None
        try:
            with open(self.registry_path, "r", encoding="utf-8") as f:
                registry_data = json.load(f)
        except FileNotFoundError:
            pass
        except Exception:
            # For safety, you could back up or rename a corrupted registry
            pass
        return registry_data

    def _write_registry(self) -> None:
        """
        Writes current registry to disk atomically.
        """
        data_json = json.dumps(self._registry, indent=2).encode("utf-8")
        _atomic_write_file(self.registry_path, data_json)

    def _read_file_bytes(self, file_path: str) -> bytes:
        with open(file_path, "rb") as f:
            return f.read()

# <Issue>: The solution violates the logging requirement by not reporting key loading, registry access, or verification failures, and lacks thread/process safety due to missing file locking during key rotation and registry updates, making it prone to race conditions and data corruption.
```